In [ ]:
# !pip install opencv-python
# !conda install -c conda-forge opencv

## 가상화
# conda create -n cv
# conda activate cv
# conda install -c conda-forge jupyterlab glob2 opencv pytest-shutil tensorflow=2.1.0
# pip install tensorflow-addons

In [2]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [3]:
PATH_DEFECT = 'data\\Defect_images\\'
PATH_MASK = 'data\\Mask_images\\'
PATH_NODEFECT = 'data\\NODefect_images\\'

### 데이터셑 불러오기

In [8]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

In [11]:
# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

### 첫번째 데이터셑 구성 (첫번째 발송 데이터)

In [12]:
if os.path.exists('data/dataset/1') is False:
    os.mkdir('data/dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, 'data/dataset/1/' + barcode + '.png')

### 두번째 데이터셑 구성

### 세번째 데이터셑 구성

### 실전 데이터셑 구성